In [ ]:
from datasets import load_dataset
import random
import re
from tqdm import tqdm
import pickle
import pandas as pd
import torch
import numpy as np
random.seed(42)


In [5]:
marco_dataset = load_dataset('ms_marco', 'v1.1')

In [6]:
with open('tockenizer.pickle', 'rb') as file:
    tokenizer = pickle.load(file)

In [7]:
def tokenize(word, tokenizer=tokenizer):
    if word in tokenizer.keys():
        return tokenizer[word]
    else:
        return 0


In [8]:
inverse_tokens = {}
def inverse(tokenizer):
    for token in tokenizer:

        inverse_tokens[tokenizer[token]] = token
#inverse(tokenizer)

In [9]:
def reverse_tokenize(word,inverse_tokens=inverse_tokens):
    if (int(word) == 0):
        return None
    return inverse_tokens[word]

In [51]:
tokenized_marco = []
for split in marco_dataset:
    #print (len(marco_dataset))
    tokenized_sample = []
    for sample in marco_dataset[split]:
        data = []
        row = []
        query_tokens = []
        #print (sample['query'])
        for word in sample['query'].split(" "):
            query_tokens.append(np.float32(tokenize(word)))
        data.append(np.float32(query_tokens))
        data.append(np.float32((sample['passages']['is_selected'])))
        #break
        for words in (sample['passages']['passage_text']):
            words_ = re.sub(r'[^a-zA-Z-\s]', '', words)
            words_ = (words_.lower().split(" "))
            tockenized_passage_text = []
            #print (words)
            for word in words_:
                #print(word)
                #print (len(word))
                tockenized_passage_text.append(np.float32(tokenize(word)))
            data.append(np.float32(tockenized_passage_text))
            #print (len(tockenized_passage_text))
        row.append((data))
        #print(row[0])
        tokenized_sample.append((row[0]))
        #print(row)
        #break
    #break
    tokenized_marco.append((tokenized_sample))

In [11]:
#print (tokenized_marco[0][0][1])
# words = []
# for word in (tokenized_marco[0][0][1]):
#     #print (int(word))
#     words.append(reverse_tokenize(int(word)))
# print ((marco_dataset['validation'][0]['passages']['passage_text'][1]).split(" "))
# print ((words))

In [79]:
tokenized_marco_val = pd.DataFrame(tokenized_marco[0]).fillna(0)
tokenized_marco_train = pd.DataFrame(tokenized_marco[1]).fillna(0)
tokenized_marco_test = pd.DataFrame(tokenized_marco[2]).fillna(0)

In [80]:
tokenized_marco_train.to_csv("tokenized_marco_train.csv")
tokenized_marco_test.to_csv("tokenized_marco_test.csv")
tokenized_marco_val.to_csv("tokenized_marco_val.csv")